In [1]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pickle
import random

In [2]:
def recommend_items(input_items, rules, top_n=5, metric='confidence'):
    recommendations = {}
    for i , (antecedent, consequent, antecedent_support, consequent_support, support, confidence, lift, leverage, conviction, zhangs_metric) in rules.iterrows():
        if antecedent.issubset(input_items) and not consequent.issubset(input_items):
            for item in consequent:
                if item not in input_items:
                    if item not in recommendations:
                        recommendations[item] = []
                    recommendations[item].append((confidence, support, lift, leverage, conviction, zhangs_metric))
    # Average the confidence and support values for the same item
    recommendations = {
        item: (
            sum(conf for conf, _, _, _, _, _ in item_rules) / len(item_rules), 
            sum(sup for _, sup, _, _, _, _ in item_rules) / len(item_rules), 
            sum(lift for _, _, lift, _, _, _ in item_rules) / len(item_rules), 
            sum(leverage for _, _, _, leverage, _, _ in item_rules) / len(item_rules),
            sum(conviction for _, _, _, _, conviction, _ in item_rules) / len(item_rules), 
            sum(zhangs_metric for _, _, _, _, _, zhangs_metric in item_rules) / len(item_rules)
                )
        for item, item_rules in recommendations.items()
    }
    
    if metric == 'confidence':
        # Sort the recommendations by decreasing confidence and support
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][0], -x[1][1]))
    elif metric == 'support':
        # Sort the recommendations by decreasing support and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][1], -x[1][0]))
    elif metric == 'lift':
        # Sort the recommendations by decreasing lift and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][2], -x[1][0]))
    elif metric == 'leverage':
        # Sort the recommendations by decreasing leverage and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][3], -x[1][0]))
    elif metric == 'conviction':
        # Sort the recommendations by decreasing conviction and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][4], -x[1][0]))
    elif metric == 'zhangs_metric':
        # Sort the recommendations by decreasing Zhang's metric and confidence
        sorted_recommendations = sorted(recommendations.items(), key=lambda x: (-x[1][5], -x[1][0]))
    else:
        raise ValueError('Invalid metric: {}'.format(metric))

    return [item for item, _ in sorted_recommendations[:top_n]]


def evaluate_recommendations(test_data, user_items, rules, top_n, metric='confidence'):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for user, true_items in tqdm(test_data.items()):
        # Assuming user_items is a dictionary with user IDs as keys and their associated items as values
        input_items = user_items[user]
        # Get recommendations for the user
        recommended_items = set(recommend_items(input_items, rules, top_n=top_n, metric=metric))
        true_items = set(true_items)
        true_positives += len(recommended_items.intersection(true_items))
        false_positives += len(recommended_items - true_items)
        false_negatives += len(true_items - recommended_items)
    # Calculate precision, recall, and F1 score
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1_score

In [4]:
# All baskets
with open('baskets.pkl', 'rb') as f:
    baskets = pickle.load(f)

# All baskets user has ever made
with open('baskets_by_user.pkl', 'rb') as f:
    user_baskets = pickle.load(f)

# All items user has ever bought
with open('items_by_user.pkl', 'rb') as f:
    user_items = pickle.load(f)

In [5]:
# Encode baskets
te = TransactionEncoder()
te_ary = te.fit(baskets).transform(baskets)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.reindex(sorted(df.columns), axis=1)  # sort columns by value
df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
14959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14960,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14961,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
def create_test_data(baskets, test_size=0.1):
    '''
    Create test data from the given baskets - drop items from the baskets and add them to the test data - see how well the model can predict them

        :param user_items: dictionary with user IDs as keys and their associated items as values
        :param test_size: size of the test data

    :return: 
        train_data - dictionary with transaction IDs as keys and their associated items as values 
        test_data - dictionary with transaction IDs as keys and their associated items as values - for testing the model (items that were left in the basket)
        test_labels - dictionary with transaction IDs as keys and their associated items as values - for testing the model (items that were dropped from the basket)
    '''

    train_data = {basket_id : items for basket_id, items in enumerate(baskets) if np.random.rand() > test_size}
    test_labels = {}
    test_data = {}

    for id, basket in enumerate(baskets):
        if id not in train_data:
            n = len(basket)
            basket = list(basket)
            if n > 1:
                # n_to_drop = np.random.randint(1, n // 2 + 1)  # drop at least one item, but not more than half of the items
                n_to_drop = 1
                items_to_drop = np.random.choice(basket, n_to_drop, replace=False)  # choose items to drop
                basket = set(basket) - set(items_to_drop)  # remove items from the basket
                test_labels[id] = set(items_to_drop)  # add items to the test data
                test_data[id] = basket  # update user items
    
    return train_data, test_data, test_labels


def encode_baskets(baskets_list):
    '''
    Encode baskets

    :param baskets: list of baskets

    :return: encoded baskets
    '''
    te = TransactionEncoder()
    te_ary = te.fit(baskets_list).transform(baskets_list)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    df = df.reindex(sorted(df.columns), axis=1)  # sort columns by value
    return df

In [7]:
train, test, test_labels = create_test_data(baskets, test_size=0.1)

In [8]:
len(train), len(test), len(test_labels), test.keys() == test_labels.keys()

(13502, 1461, 1461, True)

In [17]:
with open('train_set.pkl', 'wb') as f:
    pickle.dump(train, f)

with open('test_set.pkl', 'wb') as f:
    pickle.dump(test, f)

with open('test_labels.pkl', 'wb') as f:
    pickle.dump(test_labels, f)

In [10]:
def play_with(frequent_itemsets, metrics, test_data, test_labels, top_n=5):
    '''
    Play with different metrics and thresholds for the creation of rules

    :param frequent_itemsets: frequent itemsets 
    :param methods: dictionary of methods to use for the creation of rules mapped to their thresholds

    :return: dataframe of methods, thresholds, precision, recall, f1_score
    '''
    results = []
    if type(top_n) == list:
        for n in top_n:
            for metric, threshold_list in metrics.items():
                loop = tqdm(threshold_list)
                for threshold in loop:
                    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=threshold)
                    loop.set_description(f'Method: {metric}, Threshold: {threshold:.4f}, N_rules: {len(rules)}')
                    precision, recall, f1_score = evaluate_recommendations(test_data, test_labels, rules, top_n=n, metric=metric)
                    results.append((metric, threshold, n, precision, recall, f1_score))
        return pd.DataFrame(results, columns=['method', 'threshold', 'top_n', 'precision', 'recall', 'f1_score'])
    else:
        for metric, threshold_list in metrics.items():
            loop = tqdm(threshold_list)
            for threshold in loop:
                rules = association_rules(frequent_itemsets, metric=metric, min_threshold=threshold)
                loop.set_description(f'Method: {metric}, Threshold: {threshold:.4f}, N_rules: {len(rules)}')
                precision, recall, f1_score = evaluate_recommendations(test_data, test_labels, rules, top_n=top_n, metric=metric)
                results.append((metric, threshold, precision, recall, f1_score))
        return pd.DataFrame(results, columns=['method', 'threshold', 'precision', 'recall', 'f1_score'])

In [11]:
encoded_transactions = encode_baskets(list(train.values()))
freq_itemsets = apriori(encoded_transactions, min_support=0.0001, use_colnames=True)

methods_dict = {
    'support': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'confidence':[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1],
    'lift':[0, 0.5, 5, 10, 20, 50, 100],
    'leverage':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.001],
    'conviction':[0.5, 1, 1.1, 1.25, 1.4, 1.5, 2, 5],
    'zhangs_metric':[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1],
}

results = play_with(freq_itemsets, methods_dict, test, test_labels)

100%|██████████| 1461/1461 [00:13<00:00, 106.22it/s]  86%|████████▌ | 6/7 [17:33<01:36, 96.97s/it] 
Method: confidence, Threshold: 1.0000, N_rules: 594: 100%|██████████| 7/7 [17:47<00:00, 152.48s/it]
100%|██████████| 1461/1461 [00:13<00:00, 105.34it/s]  88%|████████▊ | 7/8 [27:29<01:45, 105.55s/it] 
Method: conviction, Threshold: 5.0000, N_rules: 595: 100%|██████████| 8/8 [27:42<00:00, 207.87s/it]
100%|██████████| 1461/1461 [00:17<00:00, 81.65it/s]589:  88%|████████▊ | 7/8 [40:52<03:52, 232.46s/it] 
Method: zhangs_metric, Threshold: 1.0000, N_rules: 589: 100%|██████████| 8/8 [41:10<00:00, 308.81s/it]


In [12]:
display(results.sort_values(by='f1_score', ascending=False).head(10))
display(results.sort_values(by='precision', ascending=False).head(10))
display(results.sort_values(by='recall', ascending=False).head(10))

,method,threshold,precision,recall,f1_score
8,confidence,0.0500,0.062552,0.203245,0.095662
2,support,0.0010,0.056607,0.159081,0.083501
3,support,0.0050,0.067102,0.083371,0.074357
7,confidence,0.0100,0.046770,0.153673,0.071714
9,confidence,0.1000,0.074092,0.068950,0.071429
1,support,0.0005,0.045368,0.139703,0.068493
4,support,0.0100,0.071429,0.030644,0.042889
42,zhangs_metric,1.0000,0.027148,0.065795,0.038436
17,lift,10.0000,0.025833,0.066697,0.037242
22,leverage,0.0002,0.023856,0.066246,0.035079


,method,threshold,precision,recall,f1_score
30,conviction,1.2500,0.200000,0.000451,0.000899
10,confidence,0.2500,0.111111,0.000451,0.000898
9,confidence,0.1000,0.074092,0.068950,0.071429
4,support,0.0100,0.071429,0.030644,0.042889
3,support,0.0050,0.067102,0.083371,0.074357
8,confidence,0.0500,0.062552,0.203245,0.095662
29,conviction,1.1000,0.058824,0.001352,0.002643
2,support,0.0010,0.056607,0.159081,0.083501
7,confidence,0.0100,0.046770,0.153673,0.071714
1,support,0.0005,0.045368,0.139703,0.068493


,method,threshold,precision,recall,f1_score
8,confidence,0.0500,0.062552,0.203245,0.095662
2,support,0.0010,0.056607,0.159081,0.083501
7,confidence,0.0100,0.046770,0.153673,0.071714
1,support,0.0005,0.045368,0.139703,0.068493
3,support,0.0050,0.067102,0.083371,0.074357
41,zhangs_metric,0.9500,0.023193,0.069401,0.034767
9,confidence,0.1000,0.074092,0.068950,0.071429
17,lift,10.0000,0.025833,0.066697,0.037242
22,leverage,0.0002,0.023856,0.066246,0.035079
42,zhangs_metric,1.0000,0.027148,0.065795,0.038436


In [13]:
encoded_transactions = encode_baskets(list(train.values()))
freq_itemsets = apriori(encoded_transactions, min_support=0.001, use_colnames=True)
methods_dict = {
    'support': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'confidence':[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1],
    'lift':[0, 0.5, 5, 10, 20, 50, 100],
    'leverage':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.001],
    'conviction':[0.5, 1, 1.1, 1.25, 1.4, 1.5, 2, 5],
    'zhangs_metric':[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1],
}
results_2 = play_with(freq_itemsets, methods_dict, test, test_labels, top_n=[1, 2, 3, 4, 5])

100%|██████████| 1461/1461 [00:00<00:00, 234613.81it/s]|████▎     | 3/7 [00:44<00:46, 11.66s/it]
Method: confidence, Threshold: 1.0000, N_rules: 0: 100%|██████████| 7/7 [00:44<00:00,  6.31s/it]
100%|██████████| 1461/1461 [00:00<00:00, 237450.23it/s]|████████▊ | 7/8 [00:40<00:02,  2.69s/it]
Method: conviction, Threshold: 5.0000, N_rules: 0: 100%|██████████| 8/8 [00:40<00:00,  5.01s/it]
100%|██████████| 1461/1461 [00:00<00:00, 245783.66it/s]62%|██████▎   | 5/8 [00:11<00:04,  1.39s/it]
Method: zhangs_metric, Threshold: 1.0000, N_rules: 0: 100%|██████████| 8/8 [00:11<00:00,  1.44s/it]
100%|██████████| 1461/1461 [00:00<00:00, 255509.24it/s]|████▎     | 3/7 [00:43<00:45, 11.33s/it]
Method: confidence, Threshold: 1.0000, N_rules: 0: 100%|██████████| 7/7 [00:43<00:00,  6.17s/it]
100%|██████████| 1461/1461 [00:00<00:00, 258494.82it/s]|██████▎   | 5/8 [00:34<00:12,  4.29s/it]
Method: conviction, Threshold: 5.0000, N_rules: 0: 100%|██████████| 8/8 [00:34<00:00,  4.27s/it]
100%|██████████| 1461/14

In [14]:
display(results_2.sort_values(by='f1_score', ascending=False).head(10))
display(results_2.sort_values(by='precision', ascending=False).head(10))
display(results_2.sort_values(by='recall', ascending=False).head(10))

,method,threshold,top_n,precision,recall,f1_score
180,confidence,0.0500,5,0.065045,0.180712,0.095658
137,confidence,0.0500,4,0.067554,0.153222,0.093767
94,confidence,0.0500,3,0.073037,0.125732,0.092399
179,confidence,0.0100,5,0.059333,0.166742,0.087522
51,confidence,0.0500,2,0.080077,0.093285,0.086178
136,confidence,0.0100,4,0.061991,0.140604,0.086045
93,confidence,0.0100,3,0.067277,0.115818,0.085113
172,support,0.0001,5,0.056607,0.159081,0.083501
173,support,0.0005,5,0.056607,0.159081,0.083501
174,support,0.0010,5,0.056607,0.159081,0.083501


,method,threshold,top_n,precision,recall,f1_score
4,support,0.010,1,0.097561,0.016224,0.027821
3,support,0.005,1,0.089163,0.029292,0.044098
9,confidence,0.100,1,0.086294,0.045967,0.059982
8,confidence,0.050,1,0.083589,0.049121,0.061879
52,confidence,0.100,2,0.081454,0.058585,0.068152
46,support,0.005,2,0.081122,0.048220,0.060486
51,confidence,0.050,2,0.080077,0.093285,0.086178
181,confidence,0.100,5,0.078963,0.059036,0.067561
138,confidence,0.100,4,0.078963,0.059036,0.067561
95,confidence,0.100,3,0.078963,0.059036,0.067561


,method,threshold,top_n,precision,recall,f1_score
180,confidence,0.0500,5,0.065045,0.180712,0.095658
179,confidence,0.0100,5,0.059333,0.166742,0.087522
172,support,0.0001,5,0.056607,0.159081,0.083501
173,support,0.0005,5,0.056607,0.159081,0.083501
174,support,0.0010,5,0.056607,0.159081,0.083501
137,confidence,0.0500,4,0.067554,0.153222,0.093767
136,confidence,0.0100,4,0.061991,0.140604,0.086045
130,support,0.0005,4,0.058017,0.131591,0.080530
131,support,0.0010,4,0.058017,0.131591,0.080530
129,support,0.0001,4,0.058017,0.131591,0.080530


In [15]:
encoded_transactions = encode_baskets(list(train.values()))
freq_itemsets = apriori(encoded_transactions, min_support=0.01, use_colnames=True)
methods_dict = {
    'support': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'confidence':[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1],
    'lift':[0, 0.5, 5, 10, 20, 50, 100],
    'leverage':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.001],
    'conviction':[0.5, 1, 1.1, 1.25, 1.4, 1.5, 2, 5],
    'zhangs_metric':[0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 1],
}
results_3 = play_with(freq_itemsets, methods_dict, test, test_labels, top_n=[1, 2, 3, 4, 5])

100%|██████████| 1461/1461 [00:00<00:00, 232786.74it/s]|████▎     | 3/7 [00:00<00:01,  3.86it/s]
Method: confidence, Threshold: 1.0000, N_rules: 0: 100%|██████████| 7/7 [00:00<00:00,  7.97it/s]
100%|██████████| 1461/1461 [00:00<00:00, 235036.75it/s]|█▎        | 1/8 [00:00<00:02,  3.05it/s]
Method: conviction, Threshold: 5.0000, N_rules: 0: 100%|██████████| 8/8 [00:00<00:00, 18.88it/s]
100%|██████████| 1461/1461 [00:00<00:00, 243701.66it/s] 0%|          | 0/8 [00:00<?, ?it/s]
Method: zhangs_metric, Threshold: 1.0000, N_rules: 0: 100%|██████████| 8/8 [00:00<00:00, 102.28it/s]
100%|██████████| 1461/1461 [00:00<00:00, 263247.62it/s]|████▎     | 3/7 [00:00<00:01,  3.90it/s]
Method: confidence, Threshold: 1.0000, N_rules: 0: 100%|██████████| 7/7 [00:00<00:00,  8.15it/s]
100%|██████████| 1461/1461 [00:00<00:00, 252821.11it/s]|█▎        | 1/8 [00:00<00:02,  3.08it/s]
Method: conviction, Threshold: 5.0000, N_rules: 0: 100%|██████████| 8/8 [00:00<00:00, 20.73it/s]
100%|██████████| 1461/1461 [00:

In [16]:
display(results_3.sort_values(by='f1_score', ascending=False).head(10))
display(results_3.sort_values(by='precision', ascending=False).head(10))
display(results_3.sort_values(by='recall', ascending=False).head(10))

,method,threshold,top_n,precision,recall,f1_score
173,support,0.0005,5,0.071429,0.030644,0.042889
143,lift,0.0000,4,0.071429,0.030644,0.042889
199,conviction,0.5000,5,0.071429,0.030644,0.042889
172,support,0.0001,5,0.071429,0.030644,0.042889
174,support,0.0010,5,0.071429,0.030644,0.042889
175,support,0.0050,5,0.071429,0.030644,0.042889
176,support,0.0100,5,0.071429,0.030644,0.042889
144,lift,0.5000,4,0.071429,0.030644,0.042889
179,confidence,0.0100,5,0.071429,0.030644,0.042889
180,confidence,0.0500,5,0.071429,0.030644,0.042889


,method,threshold,top_n,precision,recall,f1_score
9,confidence,0.1000,1,0.110294,0.013520,0.024087
95,confidence,0.1000,3,0.097701,0.015322,0.026490
181,confidence,0.1000,5,0.097701,0.015322,0.026490
52,confidence,0.1000,2,0.097701,0.015322,0.026490
138,confidence,0.1000,4,0.097701,0.015322,0.026490
8,confidence,0.0500,1,0.097561,0.016224,0.027821
1,support,0.0005,1,0.097561,0.016224,0.027821
0,support,0.0001,1,0.097561,0.016224,0.027821
7,confidence,0.0100,1,0.097561,0.016224,0.027821
2,support,0.0010,1,0.097561,0.016224,0.027821


,method,threshold,top_n,precision,recall,f1_score
173,support,0.0005,5,0.071429,0.030644,0.042889
143,lift,0.0000,4,0.071429,0.030644,0.042889
199,conviction,0.5000,5,0.071429,0.030644,0.042889
172,support,0.0001,5,0.071429,0.030644,0.042889
174,support,0.0010,5,0.071429,0.030644,0.042889
175,support,0.0050,5,0.071429,0.030644,0.042889
176,support,0.0100,5,0.071429,0.030644,0.042889
144,lift,0.5000,4,0.071429,0.030644,0.042889
179,confidence,0.0100,5,0.071429,0.030644,0.042889
180,confidence,0.0500,5,0.071429,0.030644,0.042889
